In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [ ]:
class Sentiment(BaseModel):
    """Define o sentimento e o idioma da mensagem enviada"""
    sentimento: str = Field(description="Sentimento do Texto. Deve ser 'pos', 'neg' ou 'nt' para não definido")
    lingua: str = Field(description="Língua que o texto foi escrito (deve estar no charset UTF-8)")
    
tool_sentimento = convert_to_openai_function(Sentiment)
tool_sentimento

In [ ]:
texto = "Eu gosto muito de pizza nordestina"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Analise o texto e categorize-o conforme as instruções"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = prompt | chat.bind(functions=[tool_sentimento], 
                           function_call={"name": "Sentiment"})

In [ ]:
chain.invoke({"input": texto})

In [ ]:
chain.invoke({"input": "May the Force be with you"})

### Categorização de Texto

In [ ]:
solicitacoes = [
    "Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?",
    "Não consigo acessar minha conta. A senha não está funcionando!",
    "Estou tendo dificuldades para instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar meu endereço de entrega na plataforma.",
    "O meu laptop não está ligando. Acho que é um problema no cabo de energia.",
    "Não consigo ver meus arquivos no aplicativo. Existe alguma solução?"
]

In [ ]:
from enum import Enum
from langchain.pydantic_v1 import BaseModel, Field

class SetorEnum(str, Enum):
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros"
    
class DirecionaSuporte(BaseModel):
    """Direciona a solicitação de suporte para o setor responsável"""
    setor: SetorEnum

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_direcionamento_suporte = convert_to_openai_function(DirecionaSuporte)
tool_direcionamento_suporte

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Criar o prompt com as instruções detalhadas para o modelo
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou 
outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, 
etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, 
devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento_suporte],
                            function_call={"name":"DirecionaSuporte"})
         | JsonOutputFunctionsParser())

In [ ]:
solicitacao = solicitacoes[0]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

In [ ]:
solicitacao = solicitacoes[3]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

In [ ]:
solicitacao = solicitacoes[4]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")